# 03 · API Smoke Test (Desafio_R3)

Este notebook hace un **smoke test** de la API FastAPI sin clases del proyecto **Desafio_R3**. 
 
## Instrucciones previas 
1. En una **terminal aparte**, levanta el servidor con `uv` desde la raíz del repo: 
   ```bash 
   uv sync 
   uv run fastapi dev app/api.py 
   ``` 
2. Vuelve aquí y ejecuta las celdas para probar **/health**, entrenar el modelo **/ml/train_ranker** y pedir ranking **/ml/rank_refuel_by_route**. 
 
Si ya tienes el servidor arrancado en otro puerto u host, ajusta `API_BASE` en la siguiente celda.

In [8]:
import os, time, json, textwrap 
import requests 
from pprint import pprint 
 
# Configura la URL base de tu API (ajusta si usas otro host/puerto) 
API_BASE = os.getenv("API_BASE", "http://localhost:8000") 
API_BASE

'http://localhost:8000'

## Helper: esperar a que /health esté OK

In [9]:
def wait_for_health(timeout=30): 
    """Espera a que /health responda ok o agota timeout (s).""" 
    url = f"{API_BASE}/health" 
    start = time.time() 
    last_err = None 
    while time.time() - start < timeout: 
        try: 
            r = requests.get(url, timeout=5) 
            if r.ok: 
                return r.json() 
        except Exception as e: 
            last_err = e 
        time.sleep(1) 
    raise RuntimeError(f"API no disponible en {url}. Último error: {last_err}") 
 
health = wait_for_health(15) 
pprint(health)

{'model_loaded': True, 'ok': True}


## 1) /route/plan — polilínea stub 
Probamos la generación de una polilínea entre Madrid y Barcelona (ajusta coordenadas si quieres).

In [5]:
payload_route = { 
    "route": { 
        "route_id": "R001", 
        "vehicle_id": "V001", 
        "start_lat": 40.4168, "start_lon": -3.7038, 
        "end_lat": 41.3874,   "end_lon": 2.1686, 
        "avoid_tolls": True 
    } 
} 
r = requests.post(f"{API_BASE}/route/plan", json=payload_route, timeout=30) 
print(r.status_code) 
data_route = r.json() 
print("# puntos en poly:", len(data_route.get("points", []))) 
list(data_route.keys())

200
# puntos en poly: 31


['points', 'distances_km', 'avoid_tolls']

## 2) /ml/train_ranker — entrenamiento sintético 
Entrena el pipeline **LTR (LightGBM)** o **fallback RF** si LGBM no está disponible en tu entorno. 
El modelo se guarda en `models/rank_refuel_pipe.joblib` y la API lo carga en inferencia.

In [11]:
r = requests.post(f"{API_BASE}/ml/train_ranker", params={"synthetic": True}, timeout=120) 
print(r.status_code) 
train_meta = r.json() 
pprint(train_meta)

200
{'meta': {'used': 'lightgbm'}, 'n_samples': 1000, 'ok': True}


## 3) /ml/rank_refuel_by_route — ranking de estaciones por ruta 
Solicitamos un ranking para la misma ruta. Puedes cambiar `priority` a `cost`, `balanced` o `sustainability`.

In [12]:
payload_rank = { 
    "route": payload_route["route"], 
    "liters_needed": 45, 
    "price_area_mean": 1.62, 
    "priority": "balanced" 
} 
r = requests.post(f"{API_BASE}/ml/rank_refuel_by_route", json=payload_rank, timeout=60) 
print(r.status_code) 
rank_data = r.json() 
print("used_model:", rank_data.get("used_model")) 
print("policy:", rank_data.get("policy")) 
print("# estaciones devueltas:", len(rank_data.get("stations", []))) 
print("Top 3 (resumen):") 
for i, st in enumerate(rank_data.get("stations", [])[:3], 1): 
    print(f"#{i}", st.get("brand"), 
          "price:", st.get("price_per_liter"), 
          "detour_km:", round(st.get("detour_km", 0.0), 2), 
          "wait:", st.get("wait_min"), 
          "score:", round(st.get("score", 0.0), 2))

200
used_model: True
policy: balanced
# estaciones devueltas: 1
Top 3 (resumen):
#1 Repsol price: 1.62 detour_km: 8.92 wait: 5 score: -9.67


## 4) Cambiar política por ENV (opcional) 
Puedes ajustar pesos con variables de entorno del servidor (y reiniciarlo): 
```powershell 
setx FLEET_W_CO2 0.8 
setx FLEET_SHADOW_CO2 0.25 
``` 
En Linux/macOS: `export FLEET_W_CO2=0.8` y `export FLEET_SHADOW_CO2=0.25` antes de lanzar el servidor.

## 5) (Opcional) Ingesta de puntos de carga EV 
### /ingest/ev/openchargemap (JSON) 
Sube un JSON exportado de su API y se normaliza como `data/ev_stations.csv`.

In [13]:
# Ejemplo mínimo de payload tipo OCM (simulado). 
fake_ocm = [{ 
  "ID": 123, 
  "AddressInfo": {"Title": "OCM Demo", "Latitude": 40.5, "Longitude": -3.7}, 
  "OperatorInfo": {"Title": "DemoNet"}, 
  "Connections": [ 
     {"ConnectionType": {"Title": "CCS2"}, "PowerKW": 50}, 
     {"ConnectionType": {"Title": "Type2"}, "PowerKW": 22} 
  ] 
}] 
 
files = {"file": ("ocm.json", json.dumps(fake_ocm), "application/json")} 
r = requests.post(f"{API_BASE}/ingest/ev/openchargemap", files=files, timeout=60) 
print(r.status_code) 
pprint(r.json())

200
{'ok': True, 'rows': 1}
